<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font><br> 
  <font size="5" face="Georgia">(with Maternal Effects)</font> 

</div>

In [1]:
using DataFrames,CSV,JWAS,JWAS.Datasets

INFO: Recompiling stale cache file /Users/haocheng/.julia/lib/v0.6/JWAS.ji for module JWAS.


<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multivariate Linear Additive Genetic Model</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### pedigree

In [4]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [5]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

In [6]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [8]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [9]:
model    = build_model(model_equations,R);

In [10]:
set_covariate(model,"age");

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [11]:
ped = get_pedigree(pedfile);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:01

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [12]:
set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [14]:
out = runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                         10
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





running MCMC for conventional (no markers)...  0%|      |  ETA: 0:09:42


Posterior means at iteration: 500
Residual covariance matrix: 


running MCMC for conventional (no markers)... 23%|█     |  ETA: 0:00:02

[44.9409 10.4179; 10.4179 3.18359]
Polygenic effects covariance matrix 
[96.9038 17.5241; 17.5241 5.43612]



running MCMC for conventional (no markers)... 50%|███   |  ETA: 0:00:01


Posterior means at iteration: 1000
Residual covariance matrix: 
[31.7319 7.44098; 7.44098 2.47507]
Polygenic effects covariance matrix 
[107.099 21.4449; 21.4449 6.31259]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


In [15]:
out["MCMC samples for residual covariance matrix"]

4×100 Array{Float64,2}:
 205.976    230.819    388.016   41.0797  …  22.9027   4.94777   6.16376
  40.9488    43.8722    89.2173  11.2754      5.15359  1.5393    2.61204
  40.9488    43.8722    89.2173  11.2754      5.15359  1.5393    2.61204
   8.33502    9.45403   20.6594   5.0449      1.51861  0.985724  2.03133

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. The Multivariate Linear Additive Genetic Model with Maternal Effects</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [16]:
phenofile = Datasets.dataset("testMT","maternal.txt")
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [17]:
model_equations = "BW = intercept + age + sex + Animal+ dam;
                   CW = intercept + age + sex + Animal";

In [18]:
model = build_model(model_equations,R);
set_covariate(model,"age");

In [19]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
set_random(model,"Animal dam", ped,G0)

INFO: dam is not found in model equation 2.


In [20]:
randomStr = "Animal dam"
pedTrmVec = split(randomStr," ",keep=false)  # "animal animal*age"

2-element Array{SubString{String},1}:
 "Animal"
 "dam"   

In [21]:
out=runMCMC(model,data,chain_length=1000,printout_frequency=500);

MCMC Information:

methods                        conventional (no markers)
chain_length                                   1000
starting_value                                false
printout_frequency                              500
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000





running MCMC for conventional (no markers)... 22%|█     |  ETA: 0:00:00


Posterior means at iteration: 500
Residual covariance matrix: 
[132.77 29.1637; 29.1637 7.176]
Polygenic effects covariance matrix 
[6.86738 2.30463 0.019238; 2.30463 1.64054 -0.01453; 0.019238 -0.01453 0.452579]



running MCMC for conventional (no markers)... 41%|██    |  ETA: 0:00:00


Posterior means at iteration: 1000
Residual covariance matrix: 


running MCMC for conventional (no markers)... 80%|█████ |  ETA: 0:00:00

[93.7638 20.6624; 20.6624 5.28027]
Polygenic effects covariance matrix 
[31.2868 8.36655 -0.233925; 8.36655 3.06501 -0.06686; -0.233925 -0.06686 0.485415]



running MCMC for conventional (no markers)...100%|██████| Time: 0:00:00


In [22]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 3 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of location parameters"